In [1]:
from pyspark import SparkConf, SparkContext

In [3]:
conf=SparkConf().setMaster("local").setAppName("reduction-op")
sc=SparkContext(conf=conf)

# Reduce
-'Rdd.reduce(<func>)
- 사용자가 지정하는 함수르 받아 여러개의 값을 하나로 줄여준다.

In [4]:
from operator import add

In [5]:
sample_rdd = sc.parallelize([1,2,3,4])
sample_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [6]:
sample_rdd.reduce(add)

10

파이션에 따라 결과물이 달라진다
   - 분산된 파티션들의 연산과 합치는 부분을 나눠서 생각해야 한다.

In [7]:
sample_rdd= sc.parallelize([1,2,3,4])
sample_rdd.reduce(lambda x, y : (x*2)+y)

26

In [8]:
#파티션을 1개로 지정
sc.parallelize([1,2,3,4],1).reduce(lambda x,y : (x*2)+y)

26

In [9]:
#파티션을 2개로 지정 1,2 / 3,4로 나눠짐
sc.parallelize([1,2,3,4],2).reduce(lambda x,y : (x*2)+y)

18

In [10]:
#파티션을 3개로 지정 3개일 경우는 reduce안된다. 1 | 2 | 3,4 로 나눠짐.
sc.parallelize([1,2,3,4],3).reduce(lambda x,y : (x*2)+y)

18

파티션에 데이터가 하나만 있으면 reduce가 안된다.

In [11]:
#파티션을 4로 지정 
sc.parallelize([1,2,3,4],4).reduce(lambda x,y : (x*2)+y)

26

# Fold
    -RDD.Fold(zoroValue,<func>)
    -reduce와 비슷하지만 zeroValue에 넣어놓고 싶은 시작값을 지정해서 reduce가 가능
        -zeroValue는 파티션마다 계산이 일어날 때 하니씩 더해지는 값

In [12]:
rdd= sc.parallelize([2,3,4],4)

print(rdd.reduce(lambda x,y : (x*y))) #2*3 -> *4
print(rdd.fold(1,lambda x,y : (x*y))) #1*2*3*4

24
24


In [13]:
print(rdd.reduce(lambda x,y : (x+y))) # 2+3+4
print(rdd.fold(1,lambda x,y : (x+y))) # 1+2+3+4이 아님 fold는 시작값은 파티션마다 부여해서 (1+1) + (2+1) + (3+1) + (4+1)

9
14


# GroupBy
-RDD.groupBy(<func>)
-그룹핑 함수를 받아 reduction

In [14]:
rdd=sc.parallelize([1,1,2,3,5,8])
result = rdd.groupBy(lambda x : x % 2).collect()

sorted([(x,sorted(y)) for (x,y) in result])

[(0, [2, 8]), (1, [1, 1, 3, 5])]

# Aggregate(Action)
-RDD.aggregate(zeroValue, seq0p, comb0p)
    -zeroValue : 각 파티션에서 누적할 시작 값
    -seqOp :타입 변경 함수
        -파티션 내에서 벌어지는 연산을 담당
    -combOp : 모든 결과를 하나로 합쳐주는 연산을 담당
-파티션 단위의 연산 결과를 합쳐주는 과정을 거치게 된다.

In [15]:
rdd=sc.parallelize([1,2,3,4],2)

seqOp = lambda x,y : (x[0]+y,x[1]+1) #파티션 내의 연산
combOp = (lambda x,y : (x[0]+y[0],x[1]+y[1])) #파티션의 모든 결과를 최종 연산

print(rdd.aggregate((0,0),seq0p,comb0p))

NameError: name 'seq0p' is not defined

In [ ]:
sc.stop()